In [ ]:
from sqlalchemy import create_engine

%load_ext sql
%config SqlMagic.displaylimit = None

drivername = "postgresql"
username = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
# database = "postgres"
database = "desafio04"

db_url = f"{drivername}://{username}:{password}@{host}/{database}"
engine = create_engine(db_url)

%sql engine

```sql
create database desafio04;
```

# 1. Creacion de tablas e ingreso de registros

## usuarios

```sql
%%sql
CREATE TABLE usuarios (
    id SERIAL,
    email VARCHAR,
    nombre VARCHAR,
    apellido VARCHAR,
    rol VARCHAR
    );

INSERT INTO usuarios(id, email, nombre, apellido, rol)
VALUES
(DEFAULT, 'juan@mail.com', 'juan', 'perez', 'administrador'),
(DEFAULT,'diego@mail.com', 'diego', 'munoz', 'usuario'),
(DEFAULT,'maria@mail.com', 'maria', 'meza', 'usuario'),
(DEFAULT,'roxana@mail.com','roxana', 'diaz', 'usuario'),
(DEFAULT,'pedro@mail.com', 'pedro', 'diaz', 'usuario');
```

## posts

```sql
%%sql
CREATE TABLE posts (
    id SERIAL,
    titulo VARCHAR,
    contenido TEXT,
    fecha_creacion DATE,
    fecha_actualizacion DATE,
    destacado BOOLEAN,
    usuario_id BIGINT
    );

INSERT INTO posts (
    id, titulo, contenido, fecha_creacion, fecha_actualizacion, destacado, usuario_id)
VALUES
(DEFAULT, 'prueba', 'contenido prueba', '01/01/2021', '01/02/2021', true, 1),
(DEFAULT, 'prueba2', 'contenido prueba2', '01/03/2021', '01/03/2021', true, 1),
(DEFAULT, 'ejercicios', 'contenido ejercicios', '02/05/2021', '03/04/2021', true, 2),
(DEFAULT, 'ejercicios2', 'contenido ejercicios2', '03/05/2021', '04/04/2021', false, 2),
(DEFAULT, 'random', 'contenido random', '03/06/2021', '04/05/2021', false, null);
```

## comentarios

```sql
%%sql
CREATE TABLE comentarios (
    id SERIAL,
    contenido VARCHAR,
    fecha_creacion DATE,
    usuario_id BIGINT,
    post_id BIGINT
    );

INSERT INTO comentarios (id, contenido, fecha_creacion, usuario_id, post_id)
VALUES
(DEFAULT, 'comentario 1', '03/06/2021', 1, 1),
(DEFAULT, 'comentario 2', '03/06/2021', 2, 1),
(DEFAULT, 'comentario 3', '04/06/2021', 3, 1),
(DEFAULT, 'comentario 4', '04/06/2021', 1, 2),
(DEFAULT, 'comentario 5', '04/06/2021', 2, 2);
```

# Tablas

In [ ]:
%%sql
select * from usuarios;

In [ ]:
%%sql
select * from posts;

In [ ]:
%%sql
select * from comentarios;

## 2. Cruza los datos de la tabla usuarios y posts
Mostrando las siguientes columnas: nombre y email del usuario junto al título y contenido del post.

In [ ]:
%%sql
SELECT usuarios.nombre, usuarios.email, posts.titulo, posts.contenido
FROM usuarios JOIN posts
ON usuarios.id = posts.usuario_id;

## 3. Muestra el id, título y contenido de los posts de los administradores
El administrador puede ser cualquier id y debe ser seleccionado dinámicamente.

In [ ]:
%%sql
SELECT posts.id, posts.titulo, posts.contenido
FROM usuarios JOIN posts
ON usuarios.id = posts.usuario_id
WHERE usuarios.rol = 'administrador';

## 4. Cuenta la cantidad de posts de cada usuario.
Mostrar el id y email del usuario junto con la cantidad de posts de cada usuario.

Hint: Aquí hay diferencia entre utilizar inner join, left join o right join, prueba con ambas y con eso determina cuál es la correcta. No da lo mismo desde cuál tabla partes.

In [ ]:
%%sql
SELECT usuarios.id, usuarios.email, COUNT(posts.id)
FROM usuarios LEFT JOIN posts
ON usuarios.id = posts.usuario_id
GROUP BY usuarios.id, usuarios.email
ORDER BY usuarios.id;

## 5. Muestra el email del usuario que ha creado más posts
Aquí la tabla resultante tiene un único registro y muestra solo el email.

In [ ]:
%%sql
SELECT usuarios.email
FROM posts JOIN usuarios
ON posts.usuario_id = usuarios.id
GROUP BY usuarios.id, usuarios.email
ORDER BY COUNT(posts.id) DESC;

## 6. Muestra la fecha del último post de cada usuario
Utiliza la función de agregado MAX sobre la fecha de creación.

In [ ]:
%%sql
SELECT nombre, MAX(fecha_creacion)
FROM (
    SELECT posts.contenido, posts.fecha_creacion, usuarios.nombre
    FROM usuarios JOIN posts
    ON usuarios.id = posts.usuario_id
    ) AS p
GROUP BY p.nombre;

## 7. Muestra el título y contenido del post con más comentarios

In [ ]:
%%sql
SELECT titulo, contenido
FROM posts JOIN (
    SELECT post_id, COUNT(post_id)
    FROM comentarios
    GROUP BY post_id
    ORDER BY count DESC
    LIMIT 1
    ) AS c
ON posts.id = c.post_id;

## 8. Muestra en el título y el contenido de cada post junto con el contenido de cada comentario
Asociado a los post mostrados, junto con el email del usuario que lo escribió. 

In [ ]:
%%sql
SELECT
    posts.titulo as titulo_post,
    posts.contenido as contenido_post,
    comentarios.contenido as contenido_comentario,
    usuarios.email
FROM posts JOIN comentarios
ON posts.id = comentarios.post_id
JOIN usuarios
ON comentarios.usuario_id = usuarios.id;

## 9. Muestra el contenido del último comentario de cada usuario

In [ ]:
%%sql
SELECT fecha_creacion, contenido, usuario_id
FROM comentarios as c JOIN usuarios as u
ON c.usuario_id = u.id
WHERE c.fecha_creacion = (
    SELECT MAX(fecha_creacion)
    FROM comentarios
    WHERE usuario_id = u.id
    );

# 10. Muestra los emails de los usuarios que no han escrito ningún comentario
Recuerda el Having

In [ ]:
%%sql
SELECT usuarios.email
FROM usuarios LEFT JOIN comentarios
ON usuarios.id = comentarios.usuario_id
GROUP BY usuarios.email
HAVING COUNT(comentarios.id) = 0;